In [6]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
from google.colab import files
files.upload()

Saving intents3.json to intents3.json


{'intents3.json': b'{\r\n        "intents": [\r\n                {\r\n                        "tag": "greeting",\r\n                        "patterns": [\r\n                                "Hi",\r\n                                "How are you",\r\n                                "Is anyone there?",\r\n                                "Hello",\r\n                                "Good day"\r\n                        ],\r\n                        "responses": [\r\n                                "Hello, thanks for visiting",\r\n                                "Good to see you again",\r\n                                "Hi there, how can I help?"\r\n                        ],\r\n                        "context_set": ""\r\n                },\r\n                {\r\n                        "tag": "goodbye",\r\n                        "patterns": [\r\n                                "Bye",\r\n                                "See you later",\r\n                                "Goodbye"\r\n    

In [22]:
with open ('intents3.json') as json_data:
  intents3 = json.load(json_data)

In [23]:
intents3

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Jarvis designed by Aarti.',
    'Thanks for asking. I am designed by Aarti Jadhav.',
    'I am Jarvis.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from World's l

In [24]:
words = []
classes = []
documents = []
ignore = ['?']
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


In [25]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
52 unique stemmed words ["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'favourit', 'give', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'movi', 'name', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


In [28]:
training = []
output = []
output_empty = [0] * len(classes)
for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-28-9f76819882e5>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [30]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=8, verbose=1)
model.save("model.pkl")

Epoch 1/100
176/176 [==============================] - 4s 2ms/step - loss: 1.6494 - accuracy: 0.5392
Epoch 2/100
176/176 [==============================] - 0s 2ms/step - loss: 0.7165 - accuracy: 0.9252
Epoch 3/100
176/176 [==============================] - 0s 2ms/step - loss: 0.2295 - accuracy: 0.9929
Epoch 4/100
176/176 [==============================] - 0s 2ms/step - loss: 0.0740 - accuracy: 1.0000
Epoch 5/100
176/176 [==============================] - 0s 2ms/step - loss: 0.0312 - accuracy: 1.0000
Epoch 6/100
176/176 [==============================] - 0s 2ms/step - loss: 0.0168 - accuracy: 1.0000
Epoch 7/100
176/176 [==============================] - 0s 2ms/step - loss: 0.0105 - accuracy: 1.0000
Epoch 8/100
176/176 [==============================] - 0s 2ms/step - loss: 0.0071 - accuracy: 1.0000
Epoch 9/100
176/176 [==============================] - 0s 2ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 10/100
176/176 [==============================] - 0s 2ms/step - loss: 0.0038 - accura

In [32]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [33]:
from keras.models import load_model
model = load_model("model.pkl")

In [34]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words
def bow(sentence, words):
  sentence_words = clean_up_sentence (sentence)
  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w ==s:
        bag[i] = 1
  bag=np.array(bag)
  return(bag)

In [37]:
ERROR_THRESHOLD = 0.30
def classify(sentence): 
  bag = bow(sentence, words)
  results = model.predict(np.array([bag]))
  results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]], r[1]))
    return return_list
def response(sentence):
  results = classify(sentence)
  if results:
    while results:
      for i in intents['intents']:
        if i['tag'] == results[0][0]: 
             return print(random.choice(i["responses"]))
      results.pop(0)

In [38]:
response('Where are you located?')

1/1 [==============================] - 0s 194ms/step
You can visit India to meet us


In [39]:
response('Bye')

1/1 [==============================] - 0s 48ms/step
See you later, thanks for visiting
